<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Dynamic-Schedule" data-toc-modified-id="Dynamic-Schedule-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Dynamic Schedule</a></span><ul class="toc-item"><li><span><a href="#Homogeneous-Exponential-Case" data-toc-modified-id="Homogeneous-Exponential-Case-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Homogeneous Exponential Case</a></span></li><li><span><a href="#Heterogeneous-Exponential-Case" data-toc-modified-id="Heterogeneous-Exponential-Case-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Heterogeneous Exponential Case</a></span></li><li><span><a href="#Phase-Type-Case" data-toc-modified-id="Phase-Type-Case-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Phase-Type Case</a></span><ul class="toc-item"><li><span><a href="#Phase-Type-Fit" data-toc-modified-id="Phase-Type-Fit-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Phase-Type Fit</a></span></li><li><span><a href="#Weighted-Erlang-Distribution" data-toc-modified-id="Weighted-Erlang-Distribution-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Weighted Erlang Distribution</a></span></li><li><span><a href="#Hyperexponential-Distribution" data-toc-modified-id="Hyperexponential-Distribution-1.3.3"><span class="toc-item-num">1.3.3&nbsp;&nbsp;</span>Hyperexponential Distribution</a></span></li></ul></li></ul></li></ul></div>

# Dynamic Schedule
_Roshan Mahes, Michel Mandjes, Marko Boon_

In this notebook we determine dynamic schedules that minimize the following cost function:
\begin{align*}
\omega \sum_{i=1}^{n}\mathbb{E}I_i + (1 - \omega)\sum_{i=1}^{n}\mathbb{E}W_i,\quad \omega\in(0,1),
\end{align*}
where $I_i$ and $W_i$ are the expected idle and waiting time associated to client $i$, respectively. We assume that the service tasks $B_1,\dots,B_n$ are independent and solve the problem assuming different types of distributions.

The following packages are required:

In [1]:
# math
import numpy as np
import scipy
import math
from scipy.stats import binom, erlang, poisson
from scipy.optimize import minimize

# web scraping
from urllib.request import urlopen
from bs4 import BeautifulSoup as soup
import pandas as pd

# plotting
import plotly.graph_objects as go
import plotly.express as px
from itertools import cycle

# caching
from functools import cache

## Homogeneous Exponential Case

In the first case, we assume $B_1,\dots,B_n \stackrel{i.i.d.}{\sim} B \stackrel{d}{=} \text{Exp}(\mu)$ for some $\mu > 0$. In our thesis, we have determined a recursive procedure. We state the result.

<div class="alert alert-warning">
<b>Corollary 2.5.</b>
For arrival time $t$ we have, with $X_t \sim \text{Pois}(\mu t)$ and $\ell = 2,\dots,k+1$,
\begin{align*}
p_{k1}(t) = \mathbb{P}(X_t\geq k),\quad 
p_{k\ell}(t) = \mathbb{P}(X_t = k-\ell+1).
\end{align*}
</div>

<div class="alert alert-warning">
<b>Proposition 2.7.</b>
Let $X_t \sim \text{Pois}(\mu t)$. Then
\begin{align*}
f_k(t) &= t\mathbb{P}(X_t\geq k) - \frac{k}{\mu}\mathbb{P}(X_t\geq k+1), \\
g_k(t) &= \frac{k(k-1)}{2\mu}\mathbb{P}(X_t\geq k+1) + (k-1)t\mathbb{P}(X_t\leq k-1) - \frac{\mu t^2}{2}\mathbb{P}(X_t\leq k-2).
\end{align*}
</div>

<div class="alert alert-warning">
<b>Theorem 3.5.</b>
Let $p_{k\ell}(t)$, $f_k(t)$ and $g_k(t)$ be given by Corollary 2.5 and Proposition 2.7. The following recursion holds: for $i=1,\dots,n-1$ and $k=1,\dots,i$,
\[
C_i^{\star}(k) = \inf_{t\geq 0}\left(\omega f_k(t) + (1 - \omega)g_k(t) + \sum_{\ell=1}^{k+1}p_{k\ell}(t)C_{i+1}^{\star}(\ell)\right),
\]
whereas, for $k=1,\dots,n$,
\[
C_n^{\star}(k) = (1-\omega)g_{k}(\infty) = (1-\omega)\frac{k(k-1)}{2\mu}.
\]
</div>

We have implemented the formulas as follows.

In [14]:
def cost(t,i,k,mu,omega,n,C_matrix,use_h=True):
    """
    Computes the cost of the (remaining) schedule
    when t is the next interarrival time.
    """
    
    Fk = [poisson.cdf(k,mu*t), poisson.cdf(k-2,mu*t), poisson.cdf(k-1,mu*t)]
    f = (1 - Fk[-1]) * t - (1 - Fk[0]) * k / mu
    if use_h:
        g = (k - 1) / mu
    else:
        g = Fk[-1] * (k - 1) * t - Fk[-2] * mu * t**2 / 2 + (1 - Fk[0]) * k * (k - 1) / (2 * mu)
    
    cost = omega * f + (1 - omega) * g
    cost += (1 - Fk[-1]) * Cstar_homexp(i+1,1,mu,omega,n,C_matrix,use_h)
    for l in range(2,k+2):
        cost += poisson.pmf(k-l+1,mu*t) * Cstar_homexp(i+1,l,mu,omega,n,C_matrix,use_h)
    
    return cost


def Cstar_homexp(i,k,mu=1,omega=1/2,n=15,C_matrix=None,use_h=True):
    """
    Computes C*_i(k) in the homogeneous exponential case.
    """

    if C_matrix[i-1][k-1] != None: # retrieve stored value
        pass
    elif i == n: # initial condition
        if use_h:
            C_matrix[i-1][k-1] = (1 - omega) * (k - 1) / mu
        else:
            C_matrix[i-1][k-1] = (1 - omega) * k * (k - 1) / (2 * mu)
    else:
        optimization = minimize(cost,0,args=(i,k,mu,omega,n,C_matrix,use_h),method='Nelder-Mead')
        C_matrix[i-1][k-1] = optimization.fun
        minima[i-1][k-1] = optimization.x[0]
    
    return C_matrix[i-1][k-1]


Now we plot our dynamic schedule for $n = 15$ and $\omega = 0.5$:

In [28]:
omega = 0.5
n = 15

# compute schedule
C_matrix = [[None for k in range(n+1)] for i in range(n)]
minima = [[None for k in range(n+1)] for i in range(n)]

for i in range(1,n+1):
    for k in range(1,i+1):
        Cstar_homexp(i,k,mu=1,omega=omega,n=n,C_matrix=C_matrix,use_h=True)

# plot schedule
palette = cycle(px.colors.cyclical.mrybm[2:])
fig = go.Figure()

for k in range(1,n):
    fig.add_trace(go.Scatter(x=np.arange(1,n+2), y=[minima[i][k-1] for i in range(n)],
                             name=k, marker_color=next(palette)))
                                                   
fig.update_layout(
    template='plotly_white',
    title='$\\text{Dynamic Schedule}\ (n=' + f'{n},\ \omega={omega})$',
    legend_title='$\\text{Clients in System}\ (k)$', 
    xaxis = {'title': '$\\text{Client Position}\ (i)$', 'range': [0.7, n - 0.7], 'dtick': 1},
    yaxis = {'title': '$\\text{Interarrival Time}\ (\\tau_{i}(k))$', 'dtick': 1}
)

fig.show()

print(f'Cost: {C_matrix[0][0]}')

Cost: 6.05029568250075


In [24]:
minima

[[0.8793125000000009],
 [0.8793125000000009, 1.9436250000000024],
 [0.8793125000000009, 1.9436250000000024, 2.9912500000000035],
 [0.8793125000000009,
  1.9436250000000024,
  2.9912500000000035,
  4.029437500000006],
 [0.8793125000000009,
  1.9436250000000024,
  2.9912500000000035,
  4.029437500000006,
  5.061687500000007],
 [0.8793125000000009,
  1.9436250000000024,
  2.9912500000000035,
  4.029437500000006,
  5.061687500000007,
  6.089687500000007],
 [0.8793125000000009,
  1.9436250000000024,
  2.9912500000000035,
  4.029437500000006,
  5.061687500000007,
  6.089687500000007,
  7.114687500000009],
 [0.8793125000000009,
  1.9436250000000024,
  2.9912500000000035,
  4.029437500000006,
  5.061687500000007,
  6.089687500000007,
  7.114687500000009,
  8.137250000000009],
 [0.8793125000000009,
  1.9436250000000024,
  2.9912500000000035,
  4.029437500000006,
  5.0616250000000065,
  6.089687500000007,
  7.114687500000009,
  8.137250000000009,
  9.15800000000001],
 [0.8793125000000009,
  1.94

## Heterogeneous Exponential Case

Now we consider the case that the service tasks $B_i$ are independent and _heterogeneous exponentially_ distributed, i.e. $B_i \sim \text{Exp}(\mu_i)$, $i=1,\dots,n$. For ease we assume that all $\mu_i$ are distinct, i.e., $\mu_i \neq \mu_j$ for $i,j = 1,\dots,n$, $i\neq j$, but the case that some of the $\mu_i$ coincide can be considered analogously. We obtain the following result.

<div class="alert alert-warning">
<b>Lemma 2.12.</b>
For $k=1,\dots,n$ and $\ell=0,\dots,n-k$, we can write the density $\varphi_{k\ell}$ as
\[
\varphi_{k\ell}(s) := \mathbb{P}\left(\sum_{j=k}^{k+\ell}B_j \in\mathrm{d}s\right)
= \sum_{j=k}^{k+\ell}c_{k\ell j}e^{-\mu_j s},\quad s \geq 0.
\]
The coefficients $c_{k\ell j}$ are given recursively through $c_{k0k} = \mu_k$ and
\[
c_{k,\ell+1,j} = c_{k\ell j}\frac{\mu_{k+\ell+1}}{\mu_{k+\ell+1} - \mu_j}\quad \text{for}\ j = k,\dots,k+\ell,\quad c_{k,\ell+1,k+\ell+1} = \sum_{j=k}^{k+\ell}c_{k\ell j}\frac{\mu_{k+\ell+1}}{\mu_j - \mu_{k+\ell+1}}.
\]
</div>

<div class="alert alert-warning">
<b>Proposition 2.16.</b>
For $i=1,\dots,n-1$, $k=1,\dots,i$, $\ell = 2,\dots,k+1$ and $t\geq 0$,
\[
p_{k1,i}(t) = 1 - \sum_{\ell=2}^{k+1}p_{k\ell,i}(t),\quad
p_{k\ell,i}(t) = \frac{\varphi_{i-k+1,k-\ell+1}(t)}{\mu_{i-\ell+2}}.
\]
</div>

<div class="alert alert-warning">
<b>Proposition 2.17.</b>
For $i=1,\dots,n-1$ and $k=1,\dots,i$,
\begin{align*}
f_{k,i}(t) = t - \sum_{j=i-k+1}^{i}\frac{c_{i-k+1,k-1,j}}{\mu_j}\psi_{j}(t),
\quad
g_{k,i}(t) = \sum_{\ell=0}^{k-1}(k-\ell-1)\sum_{j=i-k+1}^{i-k+\ell+1}\frac{c_{i-k+1,\ell,j}}{\mu_{i-k+\ell+1}}\psi_{j}(t),
\end{align*}
with $\psi_{j}(t) = (1 - e^{-\mu_j t})/\mu_j$.
</div>

<div class="alert alert-warning">
<b>Theorem 3.9.</b>
We can determine the $C^{\star}_i(k)$ recursively: for $i=1,\dots,n-1$ and $k=1,\dots,i$,
\[
C^{\star}_i(k) = \inf_{t\ge 0}\left(\omega f_{k,i}(t) + (1-\omega)g_{k,i}(t) + \sum_{\ell=1}^{k+1}p_{k\ell,i}(t)C^{\star}_{i+1}(\ell)\right),
\]
whereas, for $k=1,\dots,n$,
\[
C^{\star}_n(k) = (1 - \omega)g_{k,n}(\infty) = (1 - \omega)\sum_{\ell=0}^{k-1}(k-\ell-1)\frac{1}{\mu_{n-k+\ell+1}}.
\]
</div>

These formulas lead to the following implementation.

In [30]:
# helper functions
def c(k,l,j,mu):
    """Computes the weights c of phi recursively (Lemma 2.23)."""

    # storage indices
    k_, l_, j_ = k - 1, l, j - 1
    
    if c_stored[k_][l_][j_] != None:
        pass
    elif k == j and not l:
        c_stored[k_][l_][j_] = mu[k_]
    elif l:
        if j >= k and j < k + l:
            c_stored[k_][l_][j_] = c(k,l-1,j,mu) * mu[k_+l_] / (mu[k_+l_] - mu[j-1])
        elif k + l == j:
            c_stored[k_][l_][j_] = sum([c(k,l-1,m,mu) * mu[j-1] / (mu[m-1] - mu[j-1])
                                        for m in range(k,k+l)])
    
    return c_stored[k_][l_][j_]

def phi(k,l,s,mu):
    return sum([c(k,l,j,mu) * math.exp(-mu[j-1] * s) for j in range(k,k+l+1)])

def psi(j,t,mu):
    return (1 - math.exp(-mu[j-1] * t)) / mu[j-1]

# transition probabilities
def trans_prob_het(t,i,k,mu):
    """Computes the transition probabilities (Prop. 2.25)."""
    
    p = [phi(i-k+1,k-l+1,t,mu) / mu[i-l+1] for l in range(2,k+2)]
    
    return [1 - sum(p)] + p

# cost function
def cost_het(t,i,k,mu,omega,n,C_matrix,use_h=True):
    """Computes the cost of the (remaining) schedule
    when t is the next interarrival time."""
    
    f = t - sum([c(i-k+1,k-1,j,mu) * psi(j,t,mu) / mu[j-1] for j in range(i-k+1,i+1)])

    if use_h:
        g = sum(1 / mu[i-k:i-1])
    else:        
        g = 0
        for l in range(k-1):
            g += (k - l - 1) * sum([c(i-k+1,l,j,mu) * psi(j,t,mu) / mu[i-k+l] for j in range(i-k+1,i-k+l+2)])
    
    
    p = trans_prob_het(t,i,k,mu)
    cost = omega * f + (1 - omega) * g
    cost += sum([Cstar_het(i+1,l,mu,omega,n,C_matrix,use_h) * p[l-1] for l in range(1,k+2)])
    
    return cost


def Cstar_het(i,k,mu,omega,n,C_matrix,use_h=True):
    """Computes C*_i(k) in the heterogeneous exponential case."""
    
    if C_matrix[i-1][k-1] != None: # retrieve stored value
        pass
    elif i == n: # initial condition
        if use_h:
            C_matrix[i-1][k-1] = (1 - omega) * sum(1 / mu[i-k:i-1])
        else:
            C_matrix[i-1][k-1] = (1 - omega) * sum([(k - l - 1) / mu[n-k+l] for l in range(k)])
    else:
        optimization = minimize(cost_het,0,args=(i,k,mu,omega,n,C_matrix,use_h))#,bounds=((0,500),))
        C_matrix[i-1][k-1] = optimization.fun
        minima[i-1][k-1] = optimization.x[0]
    
    return C_matrix[i-1][k-1]


Again we can plot our dynamic schedule:

In [61]:
omega = 0.5
n = 11
mus = np.linspace(0.5,1.5,n)

# plot schedule
palette = cycle(px.colors.cyclical.mrybm[2:])
fig = go.Figure()

print(f'omega = {omega}\nmu = {mus}\n')

C_matrix = [[None for k in range(n)] for i in range(n)]
minima = [[None for k in range(n)] for i in range(n)]
c_stored = [[[None for j in range(n)] for l in range(n)] for k in range(n)]

# compute values
for i in range(1,n+1):
    for k in range(1,i+1):
        Cstar_het(i,k,mus,omega=omega,n=n,C_matrix=C_matrix,use_h=True)

# cost
print(f'Cost: {C_matrix[0][0]}')

for k in range(1,n):
    fig.add_trace(go.Scatter(x=np.arange(1,n+2), y=[minima[i][k-1] for i in range(n)],
                             name=k, marker_color=next(palette)))

fig.update_layout(
    template='plotly_white',
    title='$\\text{Dynamic Schedule}\ (n=' + f'{n},\ \omega={omega})$',
    legend_title='$\\text{Clients in System}\ (k)$', 
    xaxis = {'title': '$\\text{Client Position}\ (i)$', 'range': [0.7, n - 0.7], 'dtick': 1},
    yaxis = {'title': '$\\text{Interarrival Time}\ (\\tau_{i}(k))$', 'dtick': 1},
    width=800,
    height=600
)

fig.show()


omega = 0.5
mu = [0.5 0.6 0.7 0.8 0.9 1.  1.1 1.2 1.3 1.4 1.5]

Cost: 5.134931235680707


## Phase-Type Case

Our most general case consists of service time distributions constructed by convolutions and mixtures of exponential distributions, the so-called _phase-type distributions_.

### Phase-Type Fit

There are two special cases of phase-type distributions that are of particular interest: the weighted Erlang distribution and the hyperexponential distribution. The idea is to fit the first two moments of the real service-time distribution. The former distribution can be used to approximate any non-negative distribution with coefficient of variation below 1, whereas the latter can be used if this coefficient of variation is larger than 1. The parameters of the weighted Erlang and hyperexponential distribution are obtained with the following function.

In [3]:
def SCV_to_params(SCV, mean=1):
    
    # weighted Erlang case
    if SCV <= 1:
        K = math.floor(1/SCV)
        p = ((K + 1) * SCV - math.sqrt((K + 1) * (1 - K * SCV))) / (SCV + 1)
        mu = (K + 1 - p) / mean
    
        return K, p, mu
    
    # hyperexponential case
    else:
        p = 0.5 * (1 + np.sqrt((SCV - 1) / (SCV + 1)))
        mu = 1 / mean
        mu1 = 2 * p * mu
        mu2 = 2 * (1 - p) * mu
        
        return p, mu1, mu2

In the following subsections we develop procedures for finding the optimal static schedule in the weighted Erlang case and the hyperexponential case, respectively.

### Weighted Erlang Distribution

In this case, we assume that the service time $B$ equals w.p. $p\in[0,1]$ an Erlang-distributed random variable with $K$ exponentially distributed phases, each of them having mean $\mu^{-1}$, and with probability $1-p$ an Erlang-distributed random variable with $K+1$ exponentially distributed phases, again with mean $\mu^{-1}$:

\begin{align*}
B \stackrel{\text{d}}{=} \sum_{i=1}^{K}X_i + X_{K+1}\mathbb{1}_{\{U > p\}},
\end{align*}

where $X_i \stackrel{iid}{\sim} \text{Exp}(\mu)$ and $U\sim\text{Unif}[0,1]$. The following recursion can be found in the thesis.

<div class="alert alert-warning">
<b>Theorem 3.16 (discrete version).</b>
For $i=1,\dots,n-1$, $k=1,\dots,i$, and $m\in\mathbb{N}_0$,
\[
\xi_i(k,m) = \inf_{t\in \mathbb{N}_0}\Bigg(\omega \bar{f}^{\circ}_{k,m\Delta}(t\Delta) 
+ (1 - \omega)\bar{h}^{\circ}_{k,m\Delta} + \sum_{\ell=2}^{k}\sum_{j=0}^{t}\bar{q}_{k\ell,mj}(t)\xi_{i+1}(\ell,j)
+ P^{\downarrow}_{k,m\Delta}(t\Delta)\xi_{i+1}(1,0) + P^{\uparrow}_{k,m\Delta}(t\Delta)\xi_{i+1}(k+1,m+t) \Bigg),
\]
whereas, for $k=1,\dots,n$ and $m \in \mathbb{N}_0$,
\[
\xi_n(k,m) = (1 - \omega)\bar{h}^{\circ}_{k,m\Delta}.
\]
</div>

Below is our implementation.

In [4]:
### helper functions

@cache
def gamma(z, u):
    gamma_circ = poisson.pmf(z-1, mu*u)

    if z == K + 1:
        gamma_circ *= (1 - p)

    return gamma_circ / B_sf(u)

@cache
def B_sf(t):
    """The survival function P(B > t)."""
    return poisson.cdf(K-1, mu*t) + (1 - p) * poisson.pmf(K, mu*t)

@cache
def P_k0(k, z, t):
    """Computes P(N_t- = 0 | N_0 = k, Z_0 = z)."""
    if z <= K:
        return sum([binom.pmf(m, k, 1-p) * erlang.cdf(t, k*K-z+1+m, scale=1/mu) for m in range(k+1)])
    elif z == K + 1:
        return sum([binom.pmf(m, k-1, 1-p) * erlang.cdf(t, (k-1)*K+1+m, scale=1/mu) for m in range(k)])

@cache
def psi(v, t, k, l):
    """
    Computes P(t-v < Erl(k,mu) < t, Erl(k,mu) + Erl(l-k,mu) > t),
    where Erl(k,mu) and Erl(l-k,mu) are independent.
    """
    return sum([poisson.pmf(j, mu*t) * binom.sf(j-k, j, v/t) for j in range(k, l)])


@cache
def f(k, t):
    return poisson.sf(k-1, mu*t) * t - poisson.sf(k, mu*t) * k / mu

@cache
def f_bar(k, z, t):
    """Computes the mean idle time given (N_0, Z_0) = (k,z)."""
    if z <= K:
        return sum([binom.pmf(m, k, 1 - p) * f(k*K+1-z+m, t) for m in range(k+1)])
    elif z == K + 1:
        return sum([binom.pmf(m, k-1, 1 - p) * f((k-1)*K+1+m, t) for m in range(k)])

@cache
def f_circ(k, u, t):
    """Computes the mean idle time given (N_0, B_0) = (k,u)."""
    return sum([gamma(z, u) * f_bar(k, z, t) for z in range(1, K+2)])

@cache
def h_bar(k, z):
    """Computes the mean waiting time given (N_0, Z_0) = (k,z)."""
    if k == 1:
        return 0
    elif z <= K:
        return ((k - 1) * (K + 1 - p) + 1 - z) / mu
    elif z == K + 1:
        return ((k - 2) * (K + 1 - p) + 1) / mu

@cache
def h_circ(k, u):
    """Computes the mean waiting time given (N_0, B_0) = (k,u)."""
    return sum([gamma(z, u) * h_bar(k, z) for z in range(1, K+2)])


### transition probabilities

# 1. No client has been served before time t.
@cache
def P_up(k, u, t):
    """Computes P(N_t- = k | N_0 = k, B_0 = u)."""
    return B_sf(u+t) / B_sf(u)

# 2. All clients have been served before time t.
@cache
def P_down(k, u, t):
    """Computes P(N_t- = 0 | N_0 = k, B_0 = u)."""
    return sum([gamma(z, u) * P_k0(k, z, t) for z in range(1, K+2)])

# 3. Some (but not all) clients have been served before time t.
@cache
def q(diff, z, v, t):
    """
    Computes P(N_t = l, B_t < v | N_0 = k, Z_0 = z).
    Note: diff = k-l.
    """

    q = 0

    if z <= K:
        for m in range(diff+2):
            I_klmz = (diff + 1) * K - z + m + 1
            E = p * psi(v, t, I_klmz, I_klmz+K) + (1 - p) * psi(v, t, I_klmz, I_klmz+K+1)
            q += binom.pmf(m, diff+1, 1-p) * E

    elif z == K + 1:
        for m in range(diff+1):
            I_klm = diff * K + m + 1
            E = p * psi(v, t, I_klm, I_klm+K) + (1 - p) * psi(v, t, I_klm, I_klm+K+1)
            q += binom.pmf(m, diff, 1-p) * E

    return q

@cache
def q_bar(diff, m, j, t):
    """
    Approximates P(N_{t*Delta} = l, B_{t*Delta} in d(j*Delta) | N_0 = k, B_0 = m * Delta).
    Note: diff = k-l.
    """
    
    lower = min(max(0, (j - 0.5) * Delta), t*Delta)
    upper = min(max(0, (j + 0.5) * Delta), t*Delta)
    
    q_bar = sum([gamma(z, m*Delta) * (q(diff, z, upper, t*Delta) - q(diff, z, lower, t*Delta)) for z in range(1, K+2)])
    
    return q_bar
    

### cost function

@cache
def cost_we(t, i, k, m):
    """Computes (approximately) the cost when
    t/Delta is the next interarrival time."""
        
    cost = omega * f_circ(k, m*Delta, t*Delta) + (1 - omega) * h_circ(k, m*Delta)
    cost += P_down(k, m*Delta, t*Delta) * xi_we(i+1, 1, 0) + P_up(k, m*Delta, t*Delta) * xi_we(i+1, k+1, m+t) #### 
    
#     print('f_circ(k, m*Delta, t*Delta)', f_circ(k, m*Delta, t*Delta))
#     print('h_circ(k, m*Delta)', h_circ(k, m*Delta))
#     print('P_down(k, m*Delta, t*Delta)', P_down(k, m*Delta, t*Delta))
#     print('xi_we(i+1, 1, 0)', xi_we(i+1, 1, 0))
#     print('P_up(k, m*Delta, t*Delta', P_up(k, m*Delta, t*Delta))
#     print('xi_we(i+1, k+1, m+t)', xi_we(i+1, k+1, m+t))

    for l in range(2, k+1):
        for j in range(t+1):
            cost += q_bar(k-l, m, j, t) * xi_we(i+1, l, j)
    
    return cost


In [9]:
k, u = 3, 4

h_circ(k, u)

1.769113842979593

In [15]:
i = 2
k = 1
m = 1
t = 9

# cost_we(t, i, k, m)

# for t in range(1,21):
#     print(t, cost_we(t, i, k, m) - cost_we(t-1, i, k, m))

In [36]:
(1 - 0.5) * h_circ(2, 1)

0.4362059564857282

In [43]:
xi_we(3,2,10) #### 0.4362059564857282

i: 3, k: 2, m: 10


0.4362059564857282

In [16]:
i = 3
k = 2
m = 1
t = 9

(1 - omega) * h_circ(k, (m+t)*Delta)


0.4362059564857282

In [ ]:
# def xi_we(i, k, m):
#     """Implements the Weighted Erlang Case."""
    
#     # truncate time in service m
#     if m >= t_MAX:
#         m_new = t_MAX-1
#     else:
#         m_new = m
    
#     if xi_matrix[i-1][k-1][m]: # retrieve stored value
#         pass
#     elif i == n: # initial condition
#         xi_matrix[i-1][k-1][m] = (1 - omega) * h_circ(k, m*Delta)        
#     else:
        
#         # initial guess
#         if m > 0 and minima[i-1][k-1][m-1]:
#             t_guess = minima[i-1][k-1][m-1]
#         else:
#             t_guess = eval(old_minima[i-1][k-1])[m]
        
#         cost_guess = cost_we(t_guess, i, k, m)
#         t_new = t_guess
        
#         # walk to the left
#         while True:
#             t_new -= 1
#             cost_new = cost_we(t_new, i, k, m)

#             if cost_new < cost_guess:
#                 t_guess = t_new
#                 cost_guess = cost_new
#             elif cost_new > cost_guess:
#                 break
        
#         # walk to the right
#         while True:
#             t_new += 1
#             cost_new = cost_we(t_new, i, k, m)

#             if cost_new < cost_guess:
#                 t_guess = t_new
#                 cost_guess = cost_new
#             elif cost_new > cost_guess:
#                 break

#         xi_matrix[i-1][k-1][m] = cost_guess
#         minima[i-1][k-1][m] = t_guess

#         print("end",i,k,m,t_guess,cost_guess)

#     return xi_matrix[i-1][k-1][m]       


In [8]:
def xi_we(i, k, m):
    """Implements the Weighted Erlang Case."""
    
    if m <= t_MAX and xi_matrix[i-1][k-1][m]: # retrieve stored value
        pass
    elif i == n: # initial condition
        if m <= t_MAX:
            xi_matrix[i-1][k-1][m] = (1 - omega) * h_circ(k, m*Delta)
        else:
            return (1 - omega) * h_circ(k, m*Delta)
    else:
        if m <= t_MAX:
            # initial guess
            if m > 0 and minima[i-1][k-1][m-1]:
                t_guess = minima[i-1][k-1][m-1]
            else:
                t_guess = eval(old_minima[i-1][k-1])[m]
        else:
            if minima[i-1][k-1][t_MAX]:
                t_guess = minima[i-1][k-1][t_MAX]
            else:
                t_guess = old_minima[i-1][k-1][t_MAX]

        cost_guess = cost_we(t_guess, i, k, m)
        t_new = t_guess
        
        # walk to the left
        while True:
            t_new -= 1
            cost_new = cost_we(t_new, i, k, m)

            if cost_new < cost_guess:
                t_guess = t_new
                cost_guess = cost_new
            elif cost_new > cost_guess:
                break
        
        # walk to the right
        while True:
            t_new += 1
            cost_new = cost_we(t_new, i, k, m)

            if cost_new < cost_guess:
                t_guess = t_new
                cost_guess = cost_new
            elif cost_new > cost_guess:
                break

        if m <= t_MAX:            
            xi_matrix[i-1][k-1][m] = cost_guess
            minima[i-1][k-1][m] = t_guess
        else:
            return cost_guess
        
        if m <= 2:
            print("end",i,k,m,t_guess,cost_guess)

    return xi_matrix[i-1][k-1][m]       


In [5]:
SCV = 0.6

K, p, mu = SCV_to_params(SCV)

Delta = 0.01
# epsilon = 0.005
t_MAX = int(5/Delta) # int(5/Delta)
n = 5
omega = 0.5


In [6]:
import csv

C_matrix = [[None for k in range(n)] for i in range(n)]
minima = [[None for k in range(n-1)] for i in range(n-1)]

# compute values
for i in range(1,n+1):
    for k in range(1,i+1):
        Cstar_homexp(i,k,mu=1,omega=omega,n=n,C_matrix=C_matrix)

# # cost
print("\nCost:", C_matrix[0][0])

new_minima = [[[None for m in range(t_MAX+1)] for k in range(n-1)] for i in range(n-1)]

for i in range(n-1):
    for k in range(i+1):
        new_minima[i][k] = [int(round(minima[i][k],2) / Delta)] * t_MAX * 2

with open(f'SCV_1.00_omega_{omega}_minima.csv','w', newline='') as myfile:
    out = csv.writer(myfile)
    out.writerows(new_minima)

with open(f'SCV_1.00_omega_{omega:.1f}_minima.csv','r') as csvfile:
    reader = csv.reader(csvfile)
    old_minima = list(reader)


Cost: 1.6537135578121354


In [9]:
xi_matrix = [[[None for m in range(t_MAX+1)] for k in range(i+1)] for i in range(n)]
minima = [[[None for m in range(t_MAX+1)] for k in range(i+1)] for i in range(n)]

for i in np.arange(n,0,-1):
    for k in range(1,i+1):
        print("i =",i,"k =",k)
        for m in range(t_MAX+1):
            xi_we(i,k,m)

i = 5 k = 1
i = 5 k = 2
i = 5 k = 3
i = 5 k = 4
i = 5 k = 5
i = 4 k = 1
end 4 1 0 82 0.2873682111225399
end 4 1 1 81 0.28693974485785917
end 4 1 2 80 0.28650344978448095
i = 4 k = 2
end 4 2 0 182 0.9215541171307058
end 4 2 1 181 0.9180487498596738
end 4 2 2 180 0.914631314276397
i = 4 k = 3
end 4 3 0 281 1.5225754902496644
end 4 3 1 281 1.5191277971145967
end 4 3 2 280 1.5157679832191553
i = 4 k = 4
end 4 4 0 381 2.1070594237343347
end 4 4 1 381 2.1036447710622146
end 4 4 2 380 2.1003175119211934
i = 3 k = 1
end 3 1 0 94 0.6263562174481256
end 3 1 1 93 0.6258978562726628
end 3 1 2 92 0.6254298032627335
i = 3 k = 2
end 3 2 0 197 1.2736796408723197
end 3 2 1 197 1.2701612595145089
end 3 2 2 196 1.2667205384834836
i = 3 k = 3
end 3 3 0 300 1.8842402297082577
end 3 3 1 299 1.880782121602777
end 3 3 2 299 1.8774064863218618
i = 2 k = 1
end 2 1 0 95 0.9701097779289116
end 2 1 1 94 0.9696505804090179
end 2 1 2 93 0.9691816389343737
i = 2 k = 2
end 2 2 0 199 1.6185901568300645
end 2 2 1 198 1.

In [15]:
i, k, m = 5, 4, 2

print(xi_we(i,k,m))
print(minima[i-1][k-1][m])

1.4811601582524796
None


We proceed by analyzing the second case, i.e., the hyperexponential case.

### Hyperexponential Distribution

In this case the service times $B_i$ are independent and distributed as $B$, where $B$ equals with probability $p\in [0,1]$ an exponentially distributed random variable with mean $\mu_1^{-1}$, and with probability $1-p$ an exponentially distributed random variable with mean $\mu_{2}^{-1}$. The following recursion can be derived from the thesis.

<div class="alert alert-warning">
<b>Theorem 3.19 (discrete version).</b>
For $i=1,\dots,n-1$, $k=1,\dots,i$, and $m\in\mathbb{N}_0$,
\[
\xi_i(k,m) = \inf_{t\in \mathbb{N}_0}\Bigg(\omega \bar{f}^{\circ}_{k,m\Delta}(t\Delta) 
+ (1 - \omega)\bar{h}^{\circ}_{k,m\Delta} + \sum_{\ell=2}^{k}\sum_{j=0}^{t}\bar{q}_{k\ell,mj}(t)\xi_{i+1}(\ell,j)
+ P^{\downarrow}_{k,m\Delta}(t\Delta)\xi_{i+1}(1,0) + P^{\uparrow}_{k,m\Delta}(t\Delta)\xi_{i+1}(k+1,m+t) \Bigg),
\]
whereas, for $k=1,\dots,n$ and $m \in \mathbb{N}_0$,
\[
\xi_n(k,m) = (1 - \omega)\bar{h}^{\circ}_{k,m\Delta}.
\]
</div>

Below is our implementation.

In [8]:
### helper functions

# @cache
def gamma(z, u):
    if z == 1:
        return p * np.exp(-mu1 * u) / B_sf(u)
    elif z == 2:
        return (1 - p) * np.exp(-mu2 * u) / B_sf(u)

# @cache
def B_sf(t):
    return p * np.exp(-mu1 * t) + (1 - p) * np.exp(-mu2 * t) ### gamma_circ

# @cache
def zeta(alpha, t, k):
    if not k:
        return (np.exp(alpha * t) - 1) / alpha
    else:
        return ((t ** k) * np.exp(alpha * t) - k * zeta(alpha, t, k-1)) / alpha

# @cache
def rho(t,m,k):
    if not k:
        return np.exp(-mu2 * t) * (mu1 ** m) / ((mu1 - mu2) ** (m + 1)) * erlang.cdf(t, m+1, scale=1/(mu1 - mu2))
    elif not m:
        return np.exp(-mu1 * t) * (mu2 ** k) / math.factorial(k) * zeta(mu1-mu2, t, k)
    else:
        return (mu1 * rho(t, m-1, k) - mu2 * rho(t, m, k-1)) / (mu1 - mu2)

# @cache
def Psi(t,m,k):
    if not m:
        return erlang.cdf(t, k, scale=1/mu2)
    else:
        return erlang.cdf(t, m, scale=1/mu1) - mu1 * sum([rho(t, m-1, i) for i in range(k)])

# @cache
def chi(v, t, z, k, l):
    """
    Computes P(t-v < Erl(k,mu1) + Erl(l,mu2) < t, Erl(k,mu1) + Erl(l,mu2) + E(1,mu_z) > t),
    where Erl(k,mu1) and Erl(l,mu2) are independent.
    """
        
    if z == 1:
        
        if not k and l:
            return np.exp(-mu1 * t) * ((mu2) ** l) \
                        * (zeta(mu1-mu2, t, l-1) - zeta(mu1-mu2, t-v, l-1)) / math.factorial(l-1)
        elif k and not l:
            return poisson.pmf(k, mu1*t) * binom.sf(0, k, v/t)
        else:
            return mu2 * (rho(t, k, l-1) - np.exp(-mu1 * v) * rho(t-v, k, l-1))
    
    elif z == 2:
        
        if not k and l:
            return poisson.pmf(l, mu2*t) * binom.sf(0, l, v/t)
        elif k and not l:
            return np.exp(-mu2 * t) * (erlang.cdf(t, k, scale=1/(mu1-mu2)) - erlang.cdf(t-v, k, scale=1/(mu1-mu2))) \
                        * (mu1 / (mu1 - mu2)) ** k
        else:
            return mu1 * (rho(t, k-1, l) - np.exp(-mu2 * v) * rho(t-v, k-1, l))

# @cache    
def sigma(t, m, k):
    
    if not k:
        return t * erlang.cdf(t, m, scale=1/mu1) - (m / mu1) * erlang.cdf(t, m+1, scale=1/mu1)
    elif not m:
        return t * erlang.cdf(t, k, scale=1/mu2) - (k / mu2) * erlang.cdf(t, k+1, scale=1/mu2)
    else:
        return (t - k / mu2) * erlang.cdf(t, m, scale=1/mu1) - (m / mu1) * erlang.cdf(t, m+1, scale=1/mu1) \
                    + (mu1 / mu2) * sum([(k - i) * rho(t, m-1, i) for i in range(k)])

# @cache
def f_bar(k, z, t):
    """Computes the mean idle time given (N_0, Z_0) = (k,z)."""
    
    if z == 1:
        return sum([binom.pmf(m, k-1, p) * sigma(t, m+1, k-1-m) for m in range(k)])
    elif z == 2:
        return sum([binom.pmf(m, k-1, p) * sigma(t, m, k-m) for m in range(k)])

# @cache
def h_bar(k, z):
    """Computes the mean waiting time given (N_0, Z_0) = (k,z)."""
    
    if k == 1:
        return 0
    else:
        if z == 1:
            return (k-2) + (1/mu1)
        elif z == 2:
            return (k-2) + (1/mu2)

# @cache
def f_circ(k, u, t):
    """Computes the mean idle time given (N_0, B_0) = (k,u)."""
    return gamma(1, u) * f_bar(k, 1, t) + gamma(2, u) * f_bar(k, 2, t)

# @cache
def h_circ(k, u):
    """Computes the mean waiting time given (N_0, B_0) = (k,u)."""
    return gamma(1, u) * h_bar(k, 1) + gamma(2, u) * h_bar(k, 2)

    
### transition probabilities

# 1. No client has been served before time t.
# @cache
def P_up(k, u, t):
    """Computes P(N_t- = k | N_0 = k, B_0 = u)."""
    return B_sf(u + t) / B_sf(u)

# 2. All clients have been served before time t.
# @cache
def P_down(k, u, t):
    """Computes P(N_t- = 0 | N_0 = k, B_0 = u)."""
    return sum([binom.pmf(m, k-1, p) * (Psi(t, m+1, k-1-m) * gamma(1, u) \
                        + Psi(t, m, k-m) * gamma(2, u)) for m in range(k)])

# 3. Some (but not all) clients have been served before time t.
# @cache
def q(diff, z, v, t):
    """
    Computes P(N_t = l, B_t < v | N_0 = k, Z_0 = z).
    Note: diff = k-l.
    """

    if z == 1:
        return sum([binom.pmf(m, diff, p) * (p * chi(v, t, 1, m+1, diff-m) \
                        + (1 - p) * chi(v, t, 2, m+1, diff-m)) for m in range(diff+1)])
    elif z == 2:
        return sum([binom.pmf(m, diff, p) * (p * chi(v, t, 1, m, diff-m+1) \
                        + (1 - p) * chi(v, t, 2, m, diff-m+1)) for m in range(diff+1)])

# @cache
def q_bar(diff, m, j, t):
    """
    Approximates P(N_{t*Delta} = l, B_{t*Delta} in d(j*Delta) | N_0 = k, B_0 = m * Delta).
    Note: diff = k-l.
    """
    
    lower = min(max(0, (j - 0.5) * Delta), t*Delta)
    upper = min(max(0, (j + 0.5) * Delta), t*Delta)
    
    q1_low = q(diff, 1, lower, t*Delta)
    q1_upp = q(diff, 1, upper, t*Delta)
    q2_low = q(diff, 2, lower, t*Delta)
    q2_upp = q(diff, 2, upper, t*Delta)
    
    return gamma(1, m*Delta) * (q1_upp - q1_low) + gamma(2, m*Delta) * (q2_upp - q2_low)


### cost function

# @cache
def cost_he(t, i, k, m):
    """
    Computes (approximately) the cost when
    t/Delta is the next interarrival time.
    """

    cost = omega * f_circ(k, m*Delta, t*Delta) + (1 - omega) * h_circ(k, m*Delta)
    cost += P_down(k, m*Delta, t*Delta) * xi_he(i+1, 1, 0) + P_up(k, m*Delta, t*Delta) * xi_he(i+1, k+1, m+t)

    for l in range(2, k+1):
        for j in range(t+1):

            cost_diff = q_bar(k-l, m, j, t) * xi_he(i+1, l, j)
#             if cost_diff > 1e-10:
            cost += cost_diff

    return cost


In [25]:
# k = 2

# np.exp(-mu1 * t) * (mu2 ** k) / math.factorial(k) * zeta(mu1-mu2, t, k)

In [27]:
# (np.exp(-mu1 * t) * (mu2 ** k) / (mu2 - mu1) ** (k+1)) * \
#     (1 - sum([np.exp((mu1 - mu2) * t) * ((((mu2 - mu1) * t) ** i) / math.factorial(i)) for i in range(k+1)]))

In [57]:
l = 2

# chi_1[0,l]
np.exp(-mu1 * t) * ((mu2) ** l) \
                        * (zeta(mu1-mu2, t, l-1) - zeta(mu1-mu2, t-v, l-1)) / math.factorial(l-1)

0.0006259664048255367

In [58]:
(np.exp(-mu1 * t) * ((mu2 / (mu2 - mu1)) ** l)) * \
    (sum([np.exp(-(mu2-mu1)*(t-v)) * (((mu2 - mu1) * (t - v)) ** i) / math.factorial(i) for i in range(l)]) - \
    sum([np.exp(-(mu2-mu1)*t) * (((mu2 - mu1) * t) ** i) / math.factorial(i) for i in range(l)]))

0.0006259664048255289

In [19]:
f_circ(k, m*Delta, t*Delta)

0.001414313797522792

In [20]:
h_circ(k, m*Delta)

1.0

In [21]:
P_down(k, m*Delta, t*Delta)

0.020412354725205008

In [23]:
xi_he(i+1, 1, 0)

0.0

In [22]:
P_up(k, m*Delta, t*Delta)

0.8053247684339386

In [24]:
xi_he(i+1, k+1, m+t)

1.0092478449335704

In [18]:
t = 2
i = 4
k = 2 ### k > 1
m = 0

cost_he(t,i,k,m)

1.3134794439123407

In [175]:
v = 1.3
t = 2.8

z = 2
k = 4
l = 0

q(k-l,z,v,t) ### q hangt alleen af van k-l
q_bar(k-l, v, v, t)

1.4692406777096875e-10

In [176]:
np.exp(-mu2 * t) * ((mu1 ** k) / math.factorial(k-1)) * (zeta(mu2 - mu1, t, k-1) - zeta(mu2 - mu1, t-v, k-1))

0.35139047627887793

In [10]:
SCV = 2

p, mu1, mu2 = SCV_to_params(SCV)

n = 5

v = 0.05
t = 0.10

print(chi(v,t,1,1,0)) ## 0.00776 (klopt)
print(chi(v,t,1,0,1)) ## 0.02081 (FOUT) bij mij 0????

print(chi(v,t,2,0,1)) ## 0.0021 (klopt)
print(chi(v,t,2,1,0)) ## 0.0077 (klopt)

0.06735885509069447
0.019684231847361004
0.020257934213589106
0.06934130576431743


In [53]:
mu2-mu1

-1.1547005383792515

In [78]:
l = 1

np.exp(-mu1 * t) * ((mu2 / (mu1 - mu2)) ** l) * \
    (
        sum([np.exp(-(mu1-mu2)*(t-v)) * (((mu2 - mu1) * (t - v)) ** i) / math.factorial(i) for i in range(l)])) - \
        sum([np.exp(-(mu1-mu2)*t) * (((mu2 - mu1) * t) ** i) / math.factorial(i) for i in range(l)]
    )


-0.5958713301520124

In [20]:
l = 1

np.exp(-mu1 * t) * ((mu2 / (mu2 - mu1)) ** l) * \
        (1 - sum([np.exp(-(mu2-mu1)*t) * (((mu2 - mu1) * t) ** i) / math.factorial(i) for i in range(l)])) \
        - np.exp(-mu1*(t-v)) * ((mu2 / (mu2 - mu1)) ** l) * \
        (1 - sum([np.exp(-(mu2-mu1)*(t-v)) * (((mu2 - mu1) * (t - v)) ** i) / math.factorial(i) for i in range(l)]))


0.002081754501324677

In [5]:
def xi_he(i, k, m):
    """Implements the Hyperexponential Case."""
        
    # truncate time in service m
    if m >= t_MAX:
        m = t_MAX-1

    if xi_matrix[i-1][k-1][m]: # retrieve stored value
        pass
    elif i == n: # initial condition
        xi_matrix[i-1][k-1][m] = (1 - omega) * h_circ(k, m*Delta)
    else:
#         if m >= 2 and xi_matrix[i-1][k-1][m-1] and xi_matrix[i-1][k-1][m-2]:
#             # fill all coming values with current cost & minimum
#             if abs(xi_matrix[i-1][k-1][m-1] - xi_matrix[i-1][k-1][m-2]) < epsilon:
#                 xi_matrix[i-1][k-1][m:] = [xi_matrix[i-1][k-1][m-1]] * (t_MAX - (m - 1))
#                 minima[i-1][k-1][m:] = [minima[i-1][k-1][m-1]] * (t_MAX - (m - 1))

#                 print(i,k,m,"break")
#                 return xi_matrix[i-1][k-1][m]

        # initial guess
        if m > 0 and minima[i-1][k-1][m-1]:
            t_guess = minima[i-1][k-1][m-1]
        else:
            t_guess = eval(old_minima[i-1][k-1])[m]
        
        cost_guess = cost_he(t_guess, i, k, m)

        t_new = t_guess

        # walk to the left
        while True:
            t_new -= 1
            cost_new = cost_he(t_new, i, k, m)

            if cost_new < cost_guess:
                t_guess = t_new
                cost_guess = cost_new
            elif cost_new > cost_guess:
                break
        
        # walk to the right
        while True:
            t_new += 1
            cost_new = cost_he(t_new, i, k, m)

            if cost_new < cost_guess:
                t_guess = t_new
                cost_guess = cost_new
            elif cost_new > cost_guess:
                break


        xi_matrix[i-1][k-1][m] = cost_guess
        minima[i-1][k-1][m] = t_guess

        if m <= 20:
            print("end",i,k,m,t_guess,cost_guess)

    return xi_matrix[i-1][k-1][m]
    

With this program, we can obtain dynamic schedules in the hyperexponential case:

In [6]:
SCV = 2.5

p, mu1, mu2 = SCV_to_params(SCV)

Delta = 0.01
epsilon = 0.005
t_MAX = int(5/Delta)
n = 5
omega = 0.5


In [7]:
import csv

In [8]:
C_matrix = [[None for k in range(n)] for i in range(n)]
minima = [[None for k in range(n-1)] for i in range(n-1)]

# compute values
for i in range(1,n+1):
    for k in range(1,i+1):
        Cstar_homexp(i,k,mu=1,omega=omega,n=n,C_matrix=C_matrix)

# # cost
print("\nCost:", C_matrix[0][0])

new_minima = [[[None for m in range(t_MAX)] for k in range(n-1)] for i in range(n-1)]

for i in range(n-1):
    for k in range(i+1):
        new_minima[i][k] = [int(round(minima[i][k],2) / Delta)] * t_MAX * 2

with open(f'SCV_1.00_omega_{omega}_minima.csv','w', newline='') as myfile:
    out = csv.writer(myfile)
    out.writerows(new_minima)

with open(f'SCV_1.00_omega_{omega:.1f}_minima.csv','r') as csvfile:
    reader = csv.reader(csvfile)
    old_minima = list(reader)


Cost: 1.6537135578121354


In [9]:
xi_matrix = [[[None for m in range(t_MAX)] for k in range(i+1)] for i in range(n)]
minima = [[[None for m in range(t_MAX)] for k in range(i+1)] for i in range(n)]

In [13]:
# i = 3
# k = 1
# # m = 0

# # for k in np.arange(1,5):
# for m in np.arange(3):
#     print(i,k,m,xi_he(i,k,m))

3 1 0 0.8132959054568288
3 1 1 0.8180987817781893
3 1 2 0.8229816100502851


In [10]:
xi_matrix = [[[None for m in range(t_MAX)] for k in range(i+1)] for i in range(n)]
minima = [[[None for m in range(t_MAX)] for k in range(i+1)] for i in range(n)]

for i in np.arange(n,0,-1):
    for k in range(1,i+1):
        print("i =",i,"k =",k)
        for m in range(101):
            xi_he(i,k,m)

i = 5 k = 1
i = 5 k = 2
i = 5 k = 3
i = 5 k = 4
i = 5 k = 5
i = 4 k = 1
end 4 1 0 51 0.3892766899221483
end 4 1 1 51 0.3911973982481302
end 4 1 2 51 0.3931345451210404
end 4 1 3 51 0.3950881743079189
end 4 1 4 52 0.3970531180294289
end 4 1 5 52 0.3990318573873012
end 4 1 6 52 0.4010271309045299
end 4 1 7 52 0.40303897351777573
end 4 1 8 52 0.40506741788243716
end 4 1 9 52 0.4071124943256763
end 4 1 10 52 0.40917423079953014
end 4 1 11 53 0.41125219958522147
end 4 1 12 53 0.413338799081719
end 4 1 13 53 0.41544205962772235
end 4 1 14 53 0.41756199913540815
end 4 1 15 53 0.4196986329203527
end 4 1 16 53 0.42185197365599736
end 4 1 17 53 0.42402203132842287
end 4 1 18 54 0.42620767494452133
end 4 1 19 54 0.42840208923310824
end 4 1 20 54 0.4306131647817601
i = 4 k = 2
end 4 2 0 130 1.1468370277906883
end 4 2 1 130 1.1507011357785257
end 4 2 2 130 1.1545983150659638
end 4 2 3 131 1.1585229327683013
end 4 2 4 131 1.1624794775576734
end 4 2 5 131 1.1664692858596253
end 4 2 6 131 1.1704924326

In [12]:
xi_he(1,1,0)

1.7547249323015766

In [13]:
print('Function Summary')
functions = ['gamma', 'B_sf', 'zeta', 'rho', 'Psi', 'chi', 'sigma', 'f_bar', 'h_bar',
             'f_circ', 'h_circ', 'P_up', 'P_down', 'q', 'q_bar', 'cost_he']

for function in functions:
    
    info = eval(function).cache_info()
    
    print(f'{str(function):8s}: {info.hits:8d} hits\
            {info.misses:8d} misses\
            {info.hits/(info.hits + info.misses):.2%} gain')


Function Summary
gamma   :  9111836 hits                1000 misses            99.99% gain
B_sf    :    27552 hits                1464 misses            94.95% gain
zeta    :   199332 hits                3278 misses            98.38% gain
rho     :   255033 hits                4973 misses            98.09% gain
Psi     :    58020 hits                 812 misses            98.62% gain
chi     :   128622 hits              525990 misses            19.65% gain
sigma   :      328 hits                 812 misses            28.77% gain
f_bar   :    27532 hits                 484 misses            98.27% gain
h_bar   :     4990 hits                  10 misses            99.80% gain
f_circ  :     1386 hits               14008 misses            9.00% gain
h_circ  :    21548 hits                2500 misses            89.60% gain
P_up    :     1386 hits               14008 misses            9.00% gain
P_down  :     1386 hits               14008 misses            9.00% gain
q       : 17843292 hits 